# Creating an application with database system

In this exercice we will create an application relying on Postgres database. We already provide you with the application. You mission is to build a multi-container environment with a service running your app and another service running the database.

## Preliminaries

Before we start, we create the app and define which libraries we are going to use.

1) First you need to create your working folder. Next, we recommend you to create another folder called `app` where we are going to put the code.

Copy and paste the following code into a new file `app.py` inside folder `app`:

```python
import os
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

# Declare our Flask application.
app = Flask(__name__)
# We attach the database URL to our application. We are using environment
# variables.
app.config["SQLALCHEMY_DATABASE_URI"] = "postgresql+psycopg2://{dbuser}:{dbpass}@db/{dbname}".format(
    dbuser=os.environ["POSTGRES_USER"],
    dbpass=os.environ["POSTGRES_PASSWORD"],
    dbname=os.environ["POSTGRES_DB"]
)

# Finally we need to initiate a DB instance so Flask can communicate with our
# database.
db = SQLAlchemy(app)


# We are defining a simple table User which store an id and a username.
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String, unique=True, nullable=False)

    def as_dict(self):
        return {c.name: getattr(self, c.name) for c in self.__table__.columns}


# This function helps us to initialize the database by creating the table and
# inserting some datas.
def initialize_database():
    # Drop all tables in order to have a clean database on restart.
    db.drop_all()
    # Create tables schemas.
    db.create_all()
    # Add some usernames. Feel free to add or modify.
    db.session.add(User(username="Luke Skywalker"))
    db.session.add(User(username="Chewbacca"))
    db.session.add(User(username="Han Solo"))
    db.session.add(User(username="Leia Organa"))
    # Save it to actual database.
    db.session.commit()


# This is our main route.
@app.route("/")
def get_all():
    # We get all rows from table User.
    users = User.query.all()
    # We create a list of all user's username.
    users_list = [user.username for user in users]
    # We are returning a string as output:
    string_answer = f"Username in database are: {users_list}"
    return string_answer


if __name__ == "__main__":
    initialize_database()
    app.run(host="0.0.0.0")
```

Maybe this code is not completely unfamiliar to you. You may have recognized some SQLAlchemy! 🙌

Do not bother you with the details. This code is doing some extra work like defining and creating a table. But our application is simple, it outputs every usernames we have in our database:

```
Username in database are: ['Luke Skywalker', 'Chewbacca', 'Han Solo', 'Leia Organa']
```

Once you are down, your working should look like this:

```
.
└── app
    └── app.py
```

2) Create a `requirements.txt` in your working directory with the following content:

```txt
flask
flask-sqlalchemy
psycopg2-binary
```

Your working directory should look like:

```
.
├── app
│   └── app.py
└── requirements.txt
```

## Let's compose!

You could try to run this Flask application as is, using a virtual-environment. But you would quickly stumble upon several problems. First you need to define a bunch of environment variables: `POSTGRES_USER`, `POSTGRES_SERVER` and `POSTGRES_DB`. And most of all, you need to install Postgres on your machine and create the new database by yourself.

Using Docker Compose we get rid of these pains. Let's go step by step.

3) Create a `Dockerfile` inside your working folder.

Here are some hints to fill this `Dockerfile`:

- choose an appropriate Python image
- you can set the `WORKDIR` to `/app`
- the final `CMD` should be `sleep 10s; python app.py` (we add `sleep 10s` because the database take a bit of time to boot up and thus we wait for it)

If you are good, you should be able to build your image but not to run it.

4) Create a `docker-compose.yml` inside your working folder.

Your folder should look like:

```
.
├── Dockerfile
├── app
│   └── app.py
├── docker-compose.yml
└── requirements.txt
```

Let's fill this YAML together. The first lines should be:

```yaml
version: "3"
services:
    ...
```

a) Add a first service called `db`. Here some hints to succeed:

- look at an appropriate image with postgres and indicate it with `image`

b) Add a second service called `app`, where:

- we build the image from our `Dockerfile`
- you can precise to Docker that your application depends on another service with `depends_on`
- map the ports! (Flask use port `5000`)

c) Last but not least, we talked about environment variables earlier. We recommand you to do as follow: create a `.env` file in your working folder and add:

```
POSTGRES_USER=flask
POSTGRES_PASSWORD=password
POSTGRES_DB=flask
```

Finally add those two lines in both `app` and `db` services:

```yaml
  ...
    env_file:
      - .env
    ...
```

Your folder should contain:

```
.
├── .env
├── Dockerfile
├── app
│   └── app.py
├── docker-compose.yml
└── requirements.txt
```

5) Let's build and run!

You can use `docker-compose up --build` to rebuild before running containers.

If everything goes fine, you should see `Username in database are: ['Luke Skywalker', 'Chewbacca', 'Han Solo', 'Leia Organa']` on `http://localhost:5000`.

Well done! 🤘

6) BONUS – This part is optional.

To go further you can:

- create a volume in order to change the app.py as you go
- push your environment to AWS
- play with the application code